In [ ]:
include("Hamiltonian.jl")
include("mb.jl")

f = open("18SiteKH.txt","a")
writedlm(f, ["phi" "Eg"])
close(f)

la = Lattice(3,3); num = sitenum(la)

N = 100

eg = zeros(N+1); phi = zeros(N+1);
for i = 1: (N+1)
    phi[i] = 2*pi/N * (i-1)
    K = 2*sin(phi[i])*ones(3); J = cos(phi[i])*ones(3);
    ek = zeros(num*2)
    
    countk = 0
    for k1 = 0: (la.N1-1), k2 = 0: (la.N2-1)
        mb = MomBasis(k1, k2, la)
        Hk = Kitaev_T(K, k1, k2, mb, la)
        Hj = HeisenbergJ_T(J, k1, k2, mb, la)
        H = Hk + Hj
        
        #=
        println("Check zero columns")
        len = length(H.colptr)-1
        for j = 1:len
            if H.colptr[j+1]-1 < H.colptr[j]
                println("zero column: ", j)
            end
        end
        println("No zero columns")
        =#
        
        e, x = eigs(H, nev = 4, which = :SR, maxiter=10000)
        for k = 1:4
            ek[countk + k] = real(e[k])/num
        end
        countk += 4
    end
    ek = sort(ek)
    println(ek[1:4])
    eg[i] = ek[1]
    f = open("18SiteKH.txt","a")
    writedlm(f, [phi[i] eg[i]])
    close(f)
    println("Finish phi = ",i, "*2pi/N")
end
println("Finish All")